In [36]:
import random
from collections import Counter
from itertools import combinations
from IPython.display import display
import ipywidgets as widgets

In [37]:
RANKS = '2 3 4 5 6 7 8 9 10 J Q K A'.split()
SUITS = 'h d c s'.split()

def create_deck():
    return [rank + suit for rank in RANKS for suit in SUITS]

def card_prettify(card):
    rank = card[:-1]
    suit = card[-1]
    suit_symbols = {
        'h': '♥',
        'd': '♦',
        'c': '♣',
        's': '♠'
    }
    return f"{rank.upper()}{suit_symbols[suit]}"

In [38]:
def evaluate_hand(cards):
    ranks = [card[:-1] for card in cards]
    suits = [card[-1] for card in cards]
    rank_counts = Counter(ranks)
    suit_counts = Counter(suits)
    
    flush = any(count >= 5 for count in suit_counts.values())
    
    rank_values = [RANKS.index(r) for r in ranks]
    rank_values = list(set(rank_values))
    rank_values.sort()

    straight = False
    for i in range(len(rank_values) - 4):
        if rank_values[i+4] - rank_values[i] == 4:
            straight = True
            break
    if set([12, 0, 1, 2, 3]).issubset(rank_values):
        straight = True
    
    if straight and flush:
        return 8  
    elif 4 in rank_counts.values():
        return 7  
    elif sorted(rank_counts.values()) == [2,3]:
        return 6  
    elif flush:
        return 5  
    elif straight:
        return 4  
    elif 3 in rank_counts.values():
        return 3  
    elif list(rank_counts.values()).count(2) == 2:
        return 2  
    elif 2 in rank_counts.values():
        return 1  
    else:
        return 00

In [39]:
HAND_STRENGTHS = [
    "High Card", "One Pair", "Two Pair", "Three of a Kind",
    "Straight", "Flush", "Full House", "Four of a Kind", "Straight Flush"
]

In [40]:
def simulate(hole_cards, community_cards, num_opponents=1, simulations=1000):
    wins = 0
    ties = 0
    hand_distribution = [0] * 9  
    
    deck = create_deck()
    used_cards = hole_cards + community_cards
    for card in used_cards:
        deck.remove(card)
    
    for _ in range(simulations):
        random.shuffle(deck)
        opp_hands = []
        index = 0
        for _ in range(num_opponents):
            opp_hands.append([deck[index], deck[index+1]])
            index += 2
        remaining_community = 5 - len(community_cards)
        community = community_cards + deck[index:index+remaining_community]
        index += remaining_community
        
        player_best = evaluate_hand(hole_cards + community)
        opp_bests = []
        for hand in opp_hands:
            opp_best_hand = evaluate_hand(hand + community)
            opp_bests.append(opp_best_hand)
            hand_distribution[opp_best_hand] += 1
        
        max_opp = max(opp_bests)
        if player_best > max_opp:
            wins += 1
        elif player_best == max_opp:
            ties += 1
    
    win_prob = wins / simulations
    tie_prob = ties / simulations
    lose_prob = 1 - win_prob - tie_prob
    
    hand_distribution = [x / (simulations * num_opponents) for x in hand_distribution]
    
    return win_prob, tie_prob, lose_prob, hand_distribution

def simulate_potential_hands(hole_cards, community_cards, num_opponents=1, simulations=1000):
    potential_distribution = [0] * 9  
    
    deck = create_deck()
    used_cards = hole_cards + community_cards
    for card in used_cards:
        deck.remove(card)
    
    for _ in range(simulations):
        random.shuffle(deck)
        opp_hands = []
        index = 0
        for _ in range(num_opponents):
            opp_hands.append([deck[index], deck[index+1]])
            index += 2
        remaining_community = 5 - len(community_cards)
        potential_community = deck[index:index+remaining_community]
        
        opp_bests = []
        for hand in opp_hands:
            opp_best_hand = evaluate_hand(hand + community_cards + potential_community)
            opp_bests.append(opp_best_hand)
            potential_distribution[opp_best_hand] += 1
    
    potential_distribution = [x / (simulations * num_opponents) for x in potential_distribution]
    
    return potential_distribution

def poker_calculator():
    print("### Poker Probability Calculator - Texas Hold'em ###\n")
    
    hole_input = widgets.Text(
        description="Hole Cards:",
        placeholder="E.g., Ah Kd"
    )
    flop_input = widgets.Text(
        description="Flop:",
        placeholder="E.g., 7c 8s 9h"
    )
    turn_input = widgets.Text(
        description="Turn:",
        placeholder="E.g., Qd"
    )
    river_input = widgets.Text(
        description="River:",
        placeholder="E.g., Js"
    )
    
    num_opponents = widgets.IntSlider(value=1, min=1, max=9, step=1, description='Number of Opponents:')
    simulations = widgets.IntText(value=1000, description='Simulations:')
    
    calculate_button = widgets.Button(description="Calculate Odds")
    output = widgets.Output()
    
    def parse_cards(input_str):
        """Parse the input string to extract cards."""
        return input_str.strip().split() if input_str else []
    
    def on_calculate_clicked(b):
        with output:
            output.clear_output()
            hole_cards = parse_cards(hole_input.value)
            community_cards = parse_cards(flop_input.value) + parse_cards(turn_input.value) + parse_cards(river_input.value)
            if not hole_cards or len(hole_cards) != 2:
                print("Error: Please enter exactly two hole cards.")
                return
            
            win_prob, tie_prob, lose_prob, hand_distribution = simulate(hole_cards, community_cards, num_opponents.value, simulations.value)
            potential_distribution = simulate_potential_hands(hole_cards, community_cards, num_opponents.value, simulations.value)
            
            print(f"Hole Cards: {card_prettify(hole_cards[0])} {card_prettify(hole_cards[1])}")
            if community_cards:
                community_pretty = ' '.join([card_prettify(card) for card in community_cards])
                print(f"Community Cards: {community_pretty}")
            else:
                print("Community Cards: None")
            print(f"Number of Opponents: {num_opponents.value}")
            print(f"Simulations Run: {simulations.value}")
            print(f"\nEstimated Probabilities:")
            print(f"Win: {win_prob*100:.2f}%")
            print(f"Tie: {tie_prob*100:.2f}%")
            print(f"Lose: {lose_prob*100:.2f}%")
            if win_prob > 0.5:
                print("\nSuggested Action: Raise/Bet")
            elif win_prob > 0.2:
                print("\nSuggested Action: Call/Check")
            else:
                print("\nSuggested Action: Fold")
            
            print("\nOpponent Hand Distribution (Current):")
            for i, prob in enumerate(hand_distribution):
                print(f"{HAND_STRENGTHS[i]}: {prob*100:.2f}%")
            
            if len(community_cards) < 5:
                print("\nOpponent Potential Hand Distribution (Before River):")
                for i, prob in enumerate(potential_distribution):
                    print(f"{HAND_STRENGTHS[i]}: {prob*100:.2f}%")
    
    calculate_button.on_click(on_calculate_clicked)
    
    display(hole_input, flop_input, turn_input, river_input, num_opponents, simulations, calculate_button, output)

In [42]:
poker_calculator()

### Poker Probability Calculator - Texas Hold'em ###



Text(value='', description='Hole Cards:', placeholder='E.g., Ah Kd')

Text(value='', description='Flop:', placeholder='E.g., 7c 8s 9h')

Text(value='', description='Turn:', placeholder='E.g., Qd')

Text(value='', description='River:', placeholder='E.g., Js')

IntSlider(value=1, description='Number of Opponents:', max=9, min=1)

IntText(value=1000, description='Simulations:')

Button(description='Calculate Odds', style=ButtonStyle())

Output()